In [4]:
import sys
import os
import shutil
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
sys.path.append(os.path.abspath(".."))
from loader.Dataset import VideoDataset
from torch.utils.data import DataLoader
from models.transformers.encoders.vit_encoder import ViT
from models.transformers.decoders.vit_decoder import ViT_Decoder
from models.transformers.CustomTransformer import CustomizableTransformer
from utils.util import count_model_params, train_epoch,eval_model,train_model
from matplotlib import pyplot as plt
from matplotlib import patches
import torch
from utils.util import l1_and_ssim_loss_function
from torch.utils.tensorboard import SummaryWriter
from utils.loss_function import ReconstructionLoss_L1_Ssim

%load_ext autoreload
%autoreload 2
from loader.transforms import RGBNormalizer,Composition,CustomResize,RandomHorizontalFlip,RandomVerticalFlip,CustomColorJitter


general_configs={
"data_path":"/home/nfs/inf6/data/datasets/MOVi/movi_c/",
"number_of_frames_per_video":24,
"max_objects_in_scene":11,
"batch_size":64,
"device" : torch.device("cuda" if torch.cuda.is_available() else "cpu"),
"img_height":64,
"img_width":64,
"channels":3,
"learning_rate":3e-4,
"num_epochs":60,
"trainingMode":1 #0 training by bounding boxes, 1 training by masks. 
}

encoder_configs={
        "token_dim":128,
        "attn_dim":128,
        "num_heads":4,
        "mlp_size":512,
        "num_tf_layers":4
}
decoder_configs={
        "token_dim":128,
        "attn_dim":128,
        "num_heads":4,
        "mlp_size":512,
        "num_tf_layers":4
}

data_transform_config={
        "img_height":general_configs["img_height"],
        "img_width":general_configs["img_width"],
        "vFlip_probability":0.6,
        "hFlip_probability":0.6,
        "color_jitter_brightness":(0.8, 1.2),
        "color_jitter_hue":(-0.3, 0.3),
        "color_jitter_contrast":(0.6, 1.8),
        "color_jitter_saturation":(0.5, 1.5)
}
transform_composition = Composition([
                                        RGBNormalizer(),
                                        CustomResize((data_transform_config["img_height"],data_transform_config["img_width"])),
                                        RandomVerticalFlip(data_transform_config["vFlip_probability"]),
                                        RandomHorizontalFlip(data_transform_config["hFlip_probability"]),
                                        CustomColorJitter(
                                            brightness=data_transform_config["color_jitter_brightness"],
                                            hue=data_transform_config["color_jitter_hue"],
                                            contrast=data_transform_config["color_jitter_contrast"],
                                            saturation=data_transform_config["color_jitter_saturation"]
                                        )
                                    ])
#transform_composition=None
validation_dataset = VideoDataset(data_path=general_configs["data_path"],
                            split='validation',
                            number_of_frames_per_video=general_configs["number_of_frames_per_video"],
                            max_objects_in_scene=general_configs["max_objects_in_scene"],
                            halve_dataset=True,
                            is_test_dataset=False,
                            transforms=transform_composition)
valid_loader = DataLoader(dataset=validation_dataset,
                            batch_size=general_configs["batch_size"],
                            shuffle=False,
                            drop_last=True)
train_dataset = VideoDataset(data_path=general_configs["data_path"],
                            split='train',
                            number_of_frames_per_video=general_configs["number_of_frames_per_video"],
                            max_objects_in_scene=general_configs["max_objects_in_scene"],
                            halve_dataset=False,
                            is_test_dataset=False
                            )
train_loader = DataLoader(dataset=train_dataset,
                            batch_size=general_configs["batch_size"],
                            shuffle=True,
                            drop_last=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import time
start_time = time.time()
for batch in train_loader:
    print(f"Data loading time: {time.time() - start_time:.2f} seconds")


Data loading time: 52.02 seconds
Data loading time: 56.20 seconds
Data loading time: 60.47 seconds
Data loading time: 64.63 seconds
Data loading time: 68.79 seconds
Data loading time: 73.05 seconds
Data loading time: 77.44 seconds
Data loading time: 81.56 seconds
Data loading time: 85.86 seconds
Data loading time: 90.07 seconds
Data loading time: 94.27 seconds
Data loading time: 98.47 seconds
Data loading time: 102.69 seconds
Data loading time: 106.94 seconds
Data loading time: 111.26 seconds
Data loading time: 115.54 seconds
Data loading time: 119.76 seconds
Data loading time: 124.00 seconds
Data loading time: 128.20 seconds
Data loading time: 132.51 seconds
Data loading time: 136.88 seconds
Data loading time: 141.12 seconds
Data loading time: 145.28 seconds
Data loading time: 149.56 seconds
Data loading time: 153.72 seconds
Data loading time: 158.00 seconds
Data loading time: 162.27 seconds
Data loading time: 166.42 seconds
Data loading time: 170.71 seconds
Data loading time: 174.87 